# Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import model_selection, metrics, tree, ensemble

In [2]:
# Получение датафрейма
df = pd.read_csv('weather_preprocessed.csv', index_col=0)

In [3]:
# Информация о первых 5 записях
df.head()

,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,WindDir3pm_without_value
MinTemp,,,,,,,,,,,,,,,,,,,,,
13.4,22.9,0.6,5.468232,7.611178,44.0,20.0,24.0,71.0,22.0,1007.7,...,0,0,0,0,0,0,0,1,0,0
7.4,25.1,0.0,5.468232,7.611178,44.0,4.0,22.0,44.0,25.0,1010.6,...,0,0,0,0,0,0,0,0,1,0
12.9,25.7,0.0,5.468232,7.611178,46.0,19.0,26.0,38.0,30.0,1007.6,...,0,0,0,0,0,0,0,0,1,0
9.2,28.0,0.0,5.468232,7.611178,24.0,11.0,9.0,45.0,16.0,1017.6,...,0,0,0,0,0,0,0,0,0,0
17.5,32.3,1.0,5.468232,7.611178,41.0,7.0,20.0,82.0,33.0,1010.8,...,1,0,0,0,0,0,0,0,0,0


In [4]:
# Информация о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 140787 entries, 13.4 to 7.8
Columns: 120 entries, MaxTemp to WindDir3pm_without_value
dtypes: float64(15), int64(105)
memory usage: 130.0 MB


In [5]:
# Количество нулевых значений
df.isnull().sum()

MaxTemp                     0
Rainfall                    0
Evaporation                 0
Sunshine                    0
WindGustSpeed               0
                           ..
WindDir3pm_SW               0
WindDir3pm_W                0
WindDir3pm_WNW              0
WindDir3pm_WSW              0
WindDir3pm_without_value    0
Length: 120, dtype: int64

In [6]:
# Разделение признаков 
y = df['RainTomorrow'].values
df.pop('RainTomorrow')
X = df.values

In [7]:
# Разделение на обучаемую и тестовую выборки
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [8]:
# Масштабирование признаков
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Модель дерева решений

## Обучение модели

In [9]:
# Создание модели
tree_class = tree.DecisionTreeClassifier()

In [10]:
%%time
# Обучение модели
tree_class.fit(X_train, y_train)

Wall time: 4.29 s


DecisionTreeClassifier()

## Подбор гиперпараметров

In [11]:
# Текущие значения гиперпараметров у модели
tree_class.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [12]:
# подбираемые гиперпараметры
params = {
    'criterion' : ['gini', 'entropy'],
    'splitter' : ['best', 'random'],
    'min_samples_split' : [2, 4, 6, 8, 10], 
}

In [13]:
# Создание модели GridSearchCV
grid_tree_class = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), params)

In [14]:
%%time
# Обучение модели
grid_tree_class.fit(X_train, y_train)

Wall time: 4min 47s


GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_split': [2, 4, 6, 8, 10],
                         'splitter': ['best', 'random']})

In [15]:
# Оптимальная модель
grid_tree_class.best_estimator_

DecisionTreeClassifier(min_samples_split=10, splitter='random')

## Качество стандартной модели

In [16]:
# Предсказанное значения
y_pred = tree_class.predict(X_test)

In [17]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[18937  2963]
 [ 2893  3365]]
              precision    recall  f1-score   support

           0       0.87      0.86      0.87     21900
           1       0.53      0.54      0.53      6258

    accuracy                           0.79     28158
   macro avg       0.70      0.70      0.70     28158
weighted avg       0.79      0.79      0.79     28158



## Качество модели с гиперпараметрами

In [18]:
# Предсказанное значения
y_pred = grid_tree_class.predict(X_test)

In [19]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[19430  2470]
 [ 3070  3188]]
              precision    recall  f1-score   support

           0       0.86      0.89      0.88     21900
           1       0.56      0.51      0.54      6258

    accuracy                           0.80     28158
   macro avg       0.71      0.70      0.71     28158
weighted avg       0.80      0.80      0.80     28158



# Обучение композиций

## Бэггинг

### Композиция без гиперпаметров

#### Обучение модели

In [20]:
# Создание модели
bag_class = ensemble.BaggingClassifier(tree.DecisionTreeClassifier())

In [21]:
%%time
# Обучение модели
bag_class.fit(X_train, y_train)

Wall time: 38.2 s


BaggingClassifier(base_estimator=DecisionTreeClassifier())

#### Качество модели

In [22]:
# Предсказанные значения
y_pred = bag_class.predict(X_test)

In [23]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[20807  1093]
 [ 3196  3062]]
              precision    recall  f1-score   support

           0       0.87      0.95      0.91     21900
           1       0.74      0.49      0.59      6258

    accuracy                           0.85     28158
   macro avg       0.80      0.72      0.75     28158
weighted avg       0.84      0.85      0.84     28158



Качество модели композиции без гиперпараметров выше, чем у стандартной модели дерева решений.

Качество модели композиции без гиперпараметров выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4,29s

Время работы (обучения) модели дерева решений с гиперпараметрами - 4min 47s

Время работы (обучения) композиции без гиперпараметров - 38.2 s

### Композиция с гиперпараметрами у базовых алгоритмов

#### Обучение композиции

In [24]:
# Создание модели
bag_class = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(min_samples_split=10, splitter='random'))

In [25]:
%%time
# Обучение модели
bag_class.fit(X_train, y_train)

Wall time: 27.2 s


BaggingClassifier(base_estimator=DecisionTreeClassifier(min_samples_split=10,
                                                        splitter='random'))

#### Качество модели

In [26]:
# Предсказанные значения
y_pred = bag_class.predict(X_test)

In [27]:
# Характеристическик матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[20686  1214]
 [ 3064  3194]]
              precision    recall  f1-score   support

           0       0.87      0.94      0.91     21900
           1       0.72      0.51      0.60      6258

    accuracy                           0.85     28158
   macro avg       0.80      0.73      0.75     28158
weighted avg       0.84      0.85      0.84     28158



Качество модели композиции с гиперпараметрами у базовых моделей выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами у базовых моделей выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4,29s

Время работы (обучения) модели дерева решений с гиперпараметрами - 4min 47s

Время работы (обучения) композиции с гиперпараметрами у базовых моделей - 27.2 s

### Композиция с гиперпараметрами

#### Подбор гиперпараметров для композиции

In [28]:
# Создание модели
bag_class = ensemble.BaggingClassifier(tree.DecisionTreeClassifier())

In [29]:
# Текущие значения гиперпараметров модели
bag_class.get_params()

{'base_estimator__ccp_alpha': 0.0,
 'base_estimator__class_weight': None,
 'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': None,
 'base_estimator__max_features': None,
 'base_estimator__max_leaf_nodes': None,
 'base_estimator__min_impurity_decrease': 0.0,
 'base_estimator__min_impurity_split': None,
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 2,
 'base_estimator__min_weight_fraction_leaf': 0.0,
 'base_estimator__presort': 'deprecated',
 'base_estimator__random_state': None,
 'base_estimator__splitter': 'best',
 'base_estimator': DecisionTreeClassifier(),
 'bootstrap': True,
 'bootstrap_features': False,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [30]:
# Подбираемые гиперпараметры
params = {
    'n_estimators' : [10, 15, 20, 25],
}

In [31]:
# Создание модели GridSearchCV
grid_bag_class = model_selection.GridSearchCV(bag_class, params)

In [32]:
%%time
# Обучение модели
grid_bag_class.fit(X_train, y_train)

Wall time: 16min 10s


GridSearchCV(estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier()),
             param_grid={'n_estimators': [10, 15, 20, 25]})

In [33]:
# Оптимальная модель
grid_bag_class.best_estimator_

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=25)

#### Оценка качества

In [34]:
# Предсказанные значения
y_pred = grid_bag_class.predict(X_test)

In [35]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[20686  1214]
 [ 2953  3305]]
              precision    recall  f1-score   support

           0       0.88      0.94      0.91     21900
           1       0.73      0.53      0.61      6258

    accuracy                           0.85     28158
   macro avg       0.80      0.74      0.76     28158
weighted avg       0.84      0.85      0.84     28158



Качество модели композиции с гиперпараметрами выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4,29s

Время работы (обучения) модели дерева решений с гиперпараметрами - 4min 47s

Время работы (обучения) композиции с гиперпараметрами - 16min 10s

### Композиция с гиперпараметрами и с гиперпараметрами у базовых моделей

#### Обучение композиции

In [37]:
# Создание модели
bag_class = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(min_samples_split=10, splitter='random'), 
                                       n_estimators=25)

In [38]:
%%time
# Обучение модели
bag_class.fit(X_train, y_train)

Wall time: 1min 4s


BaggingClassifier(base_estimator=DecisionTreeClassifier(min_samples_split=10,
                                                        splitter='random'),
                  n_estimators=25)

#### Качество модели

In [39]:
# Предсказанные значения
y_pred = bag_class.predict(X_test)

In [40]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[20841  1059]
 [ 3041  3217]]
              precision    recall  f1-score   support

           0       0.87      0.95      0.91     21900
           1       0.75      0.51      0.61      6258

    accuracy                           0.85     28158
   macro avg       0.81      0.73      0.76     28158
weighted avg       0.85      0.85      0.84     28158



Качество модели композиции с гиперпараметрами и с гиперпаметрами у базовых моделей выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами и с гиперпаметрами у базовых моделей выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4,29s

Время работы (обучения) модели дерева решений с гиперпараметрами - 4min 47s

Время работы (обучения) композиции с гиперпараметрами и с гиперпаметрами у базовых моделей - 1min 4s

## Бустинг

### Композиция без гиперпараметров

#### Обучение композиции

In [45]:
# Создание модели
grad_boost_class = xgb.XGBRFClassifier()

In [46]:
%%time
# Обучение модели
grad_boost_class.fit(X_train, y_train)

C:\Users\vano_\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:24:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 8.18 s


XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=1, gamma=0, gpu_id=-1, importance_type='gain',
                interaction_constraints='', max_delta_step=0, max_depth=6,
                min_child_weight=1, missing=nan, monotone_constraints='()',
                n_estimators=100, n_jobs=8, num_parallel_tree=100,
                objective='binary:logistic', random_state=0, reg_alpha=0,
                scale_pos_weight=1, tree_method='exact', validate_parameters=1,
                verbosity=None)

#### Качество модели

In [49]:
# Предсказанные значения
y_pred = grad_boost_class.predict(X_test)

In [50]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[20976   924]
 [ 3499  2759]]
              precision    recall  f1-score   support

           0       0.86      0.96      0.90     21900
           1       0.75      0.44      0.56      6258

    accuracy                           0.84     28158
   macro avg       0.80      0.70      0.73     28158
weighted avg       0.83      0.84      0.83     28158



Качество модели композиции без гиперпараметров выше, чем у стандартной модели дерева решений.

Качество модели композиции без гиперпараметров выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4,29s

Время работы (обучения) модели дерева решений с гиперпараметрами - 4min 47s

Время работы (обучения) композиции без гиперпараметров - 8.18 s

### Композиция с гиперпараметрами

In [51]:
# Создание модели
grad_boost_class = xgb.XGBRFClassifier()

In [52]:
# Текущие значения гиперпараметров модели
grad_boost_class.get_params()

{'colsample_bynode': 0.8,
 'learning_rate': 1,
 'reg_lambda': 1e-05,
 'subsample': 0.8,
 'use_label_encoder': True,
 'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'scale_pos_weight': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [61]:
# Подбираемые гиперпараметры
params = {
    'colsample_bynode' : [0.8, 0.9, 1.0],
    'reg_lambda' : [1e-05, 1e-06, 1e-07],
    'subsample' : [0.8, 0.9]
}

In [62]:
grid_grad_boost_class = model_selection.GridSearchCV(grad_boost_class, params)

In [63]:
%%time
# Обучение модели
grid_grad_boost_class.fit(X_train, y_train)

C:\Users\vano_\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:39:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:39:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:39:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:40:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[01:42:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:42:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[01:46:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:46:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:46:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:46:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[01:49:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:50:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:50:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:50:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

GridSearchCV(estimator=XGBRFClassifier(base_score=None, booster=None,
                                       colsample_bylevel=None,
                                       colsample_bytree=None, gamma=None,
                                       gpu_id=None, importance_type='gain',
                                       interaction_constraints=None,
                                       max_delta_step=None, max_depth=None,
                                       min_child_weight=None, missing=nan,
                                       monotone_constraints=None,
                                       n_estimators=100, n_jobs=None,
                                       num_parallel_tree=None,
                                       objective='binary:logistic',
                                       random_state=None, reg_alpha=None,
                                       scale_pos_weight=None, tree_method=None,
                                       validate_parameters=None,
           

In [65]:
# Оптимальная модель
grid_grad_boost_class.best_estimator_

XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=0.9, colsample_bytree=1, gamma=0, gpu_id=-1,
                importance_type='gain', interaction_constraints='',
                max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=100, n_jobs=8,
                num_parallel_tree=100, objective='binary:logistic',
                random_state=0, reg_alpha=0, scale_pos_weight=1,
                tree_method='exact', validate_parameters=1, verbosity=None)

#### Качество модели

In [66]:
# Предсказанные значения
y_pred = grid_grad_boost_class.predict(X_test)

In [67]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[20953   947]
 [ 3480  2778]]
              precision    recall  f1-score   support

           0       0.86      0.96      0.90     21900
           1       0.75      0.44      0.56      6258

    accuracy                           0.84     28158
   macro avg       0.80      0.70      0.73     28158
weighted avg       0.83      0.84      0.83     28158



Качество модели композиции с гиперпараметрами выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4,29s

Время работы (обучения) модели дерева решений с гиперпараметрами - 4min 47s

Время работы (обучения) композиции с гиперпараметрами - 11min 34s

## Стекинг

### Композиция без гиперпараметров

#### Обучение модели

In [68]:
# Создание модели
stack_class = ensemble.StackingClassifier([('dtc', tree.DecisionTreeClassifier())])

In [69]:
%%time
# Обучение модели
stack_class.fit(X_train, y_train)

Wall time: 21.3 s


StackingClassifier(estimators=[('dtc', DecisionTreeClassifier())])

#### Качество модели

In [70]:
# Предсказанные значения
y_pred = stack_class.predict(X_test)

In [71]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[18900  3000]
 [ 2887  3371]]
              precision    recall  f1-score   support

           0       0.87      0.86      0.87     21900
           1       0.53      0.54      0.53      6258

    accuracy                           0.79     28158
   macro avg       0.70      0.70      0.70     28158
weighted avg       0.79      0.79      0.79     28158



Качество модели композиции без гиперпараметров выше, чем у стандартной модели дерева решений.

Качество модели композиции без гиперпараметров ниже, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4,29s

Время работы (обучения) модели дерева решений с гиперпараметрами - 4min 47s

Время работы (обучения) композиции без гиперпараметров - 21.3 s

### Композиция с гиперпараметрами у базовых моделей

#### Обучение модели

In [72]:
# Создание модели
stack_class = ensemble.StackingClassifier([('dtc', tree.DecisionTreeClassifier(min_samples_split=10, splitter='random'))])

In [73]:
%%time
# Обучение модели
stack_class.fit(X_train, y_train)

Wall time: 15.1 s


StackingClassifier(estimators=[('dtc',
                                DecisionTreeClassifier(min_samples_split=10,
                                                       splitter='random'))])

#### Качество модели

In [74]:
# Предсказанные значения
y_pred = stack_class.predict(X_test)

In [75]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[20337  1563]
 [ 3580  2678]]
              precision    recall  f1-score   support

           0       0.85      0.93      0.89     21900
           1       0.63      0.43      0.51      6258

    accuracy                           0.82     28158
   macro avg       0.74      0.68      0.70     28158
weighted avg       0.80      0.82      0.80     28158



Качество модели композиции с гиперпараметрами у базовых моделей ниже, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами у базовых моделей ниже, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4,29s

Время работы (обучения) модели дерева решений с гиперпараметрами - 4min 47s

Время работы (обучения) композиции с гиперпараметрами у базовых моделей - 15.1 s

### Композиция с гиперпараметрами

#### Обучение модели

In [77]:
# Создание модели
stack_class = ensemble.StackingClassifier([('dtc', tree.DecisionTreeClassifier())])

In [78]:
# Текущие значения гиперпараметров модели
stack_class.get_params()

{'cv': None,
 'estimators': [('dtc', DecisionTreeClassifier())],
 'final_estimator': None,
 'n_jobs': None,
 'passthrough': False,
 'stack_method': 'auto',
 'verbose': 0,
 'dtc': DecisionTreeClassifier(),
 'dtc__ccp_alpha': 0.0,
 'dtc__class_weight': None,
 'dtc__criterion': 'gini',
 'dtc__max_depth': None,
 'dtc__max_features': None,
 'dtc__max_leaf_nodes': None,
 'dtc__min_impurity_decrease': 0.0,
 'dtc__min_impurity_split': None,
 'dtc__min_samples_leaf': 1,
 'dtc__min_samples_split': 2,
 'dtc__min_weight_fraction_leaf': 0.0,
 'dtc__presort': 'deprecated',
 'dtc__random_state': None,
 'dtc__splitter': 'best'}

In [79]:
# Подбираемые гиперпараметры
params = {
    'stack_method' : ['auto', 'predict_proba']
}

In [80]:
# Создание модели GridSearchCV
grid_stack_class = model_selection.GridSearchCV(stack_class, params)

In [81]:
%%time
# Обучение модели
grid_stack_class.fit(X_train, y_train)

Wall time: 3min 22s


GridSearchCV(estimator=StackingClassifier(estimators=[('dtc',
                                                       DecisionTreeClassifier())]),
             param_grid={'stack_method': ['auto', 'predict_proba']})

In [82]:
# Оптимальная модель
grid_stack_class.best_estimator_

StackingClassifier(estimators=[('dtc', DecisionTreeClassifier())])

#### Качество модели

In [83]:
# Предсказанные значения
y_pred = grid_stack_class.predict(X_test)

In [84]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[18952  2948]
 [ 2911  3347]]
              precision    recall  f1-score   support

           0       0.87      0.87      0.87     21900
           1       0.53      0.53      0.53      6258

    accuracy                           0.79     28158
   macro avg       0.70      0.70      0.70     28158
weighted avg       0.79      0.79      0.79     28158



Качество модели композиции с гиперпараметрами выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами ниже, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4,29s

Время работы (обучения) модели дерева решений с гиперпараметрами - 4min 47s

Время работы (обучения) композиции с гиперпараметрами - 3min 22s

### Композиция с гиперпараметрами и с гиперпараметрами у базовых моделей

#### Обучение модели

In [86]:
# Создание модели
stack_class = ensemble.StackingClassifier([('dtc', tree.DecisionTreeClassifier(min_samples_split=10, splitter='random'))])

In [87]:
%%time
# Обучение модели
stack_class.fit(X_train, y_train)

Wall time: 15.4 s


StackingClassifier(estimators=[('dtc',
                                DecisionTreeClassifier(min_samples_split=10,
                                                       splitter='random'))])

#### Качество модели

In [88]:
# Предсказанные значения
y_pred = stack_class.predict(X_test)

In [89]:
# Характеристические матрицы
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

[[20274  1626]
 [ 3601  2657]]
              precision    recall  f1-score   support

           0       0.85      0.93      0.89     21900
           1       0.62      0.42      0.50      6258

    accuracy                           0.81     28158
   macro avg       0.73      0.68      0.69     28158
weighted avg       0.80      0.81      0.80     28158



Качество модели композиции с гиперпараметрами и с гиперпараметрами у базовых моделей выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами и с гиперпараметрами у базовых моделей ниже, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4,29s

Время работы (обучения) модели дерева решений с гиперпараметрами - 4min 47s

Время работы (обучения) композиции с гиперпараметрами и с гиперпараметрами у базовых моделей - 15.4 s